http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml

In [34]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [35]:
url = "http://info.nec.go.kr/bizcommon/selectbox/selectbox_cityCodeBySgJson.json?electionId=0020220601&electionCode=3"

In [36]:

response = requests.post(url)
response

<Response [200]>

In [37]:
city_list = response.json()["jsonResult"]["body"]
city_dict = {}
for c in city_list:
    city_dict[c["CODE"]] = c["NAME"]
    
city_dict

{1100: '서울특별시',
 2600: '부산광역시',
 2700: '대구광역시',
 2800: '인천광역시',
 2900: '광주광역시',
 3000: '대전광역시',
 3100: '울산광역시',
 5100: '세종특별자치시',
 4100: '경기도',
 4200: '강원도',
 4300: '충청북도',
 4400: '충청남도',
 4500: '전라북도',
 4600: '전라남도',
 4700: '경상북도',
 4800: '경상남도',
 4900: '제주특별자치도'}

In [38]:
pd.DataFrame(city_list)

,CODE,NAME
0,1100,서울특별시
1,2600,부산광역시
2,2700,대구광역시
3,2800,인천광역시
4,2900,광주광역시
5,3000,대전광역시
6,3100,울산광역시
7,5100,세종특별자치시
8,4100,경기도
9,4200,강원도


In [39]:
url_xhtml = "http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml"
params = "electionId=0020220601&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020220601%2Fcp%2Fcpri03.jsp&topMenuId=CP&secondMenuId=CPRI03&menuId=CPRI03&statementId=CPRI03_%233&electionCode=3&cityCode=1100&proportionalRepresentationCode=0&townCode=-1&dateCode=0&x=73&y=23"
url_xhtml

'http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml'

In [40]:
response = requests.post(url_xhtml, params=params)

In [46]:
html = bs(response.text)
table = html.select(".table01")[0]

In [48]:
pd.read_html(response.text)[0]

선거구명  사진 기호   소속정당명          성명(한자) 성별         생년월일(연령)  \
    선거구명  사진 기호   소속정당명          성명(한자) 성별         생년월일(연령)   
0  서울특별시 NaN  1  더불어민주당  송영길(宋永吉)  공약보기  남  1963.03.21(59세)   
1  서울특별시 NaN  2    국민의힘  오세훈(吳世勲)  공약보기  남  1961.01.04(61세)   
2  서울특별시 NaN  3     정의당  권수정(權秀靜)  공약보기  여  1973.08.30(48세)   
3  서울특별시 NaN  4   기본소득당  신지혜(申智惠)  공약보기  여  1987.10.22(34세)   
4  서울특별시 NaN  5     무소속  김광종(金光宗)  공약보기  남  1962.07.25(59세)   

                  주소      직업                           학력  \
                  주소      직업                           학력   
0    서울특별시 송파구 거마로3길     변호사           연세대학교 상경대학 경영학과 졸업   
1    서울특별시 광진구 능동로4길  서울특별시장         고려대학교 법학대학원 졸업(법학박사)   
2  서울특별시 강서구 강서로56나길   서울시의원                충남대학교 물리학과 졸업   
3   서울특별시 은평구 통일로72길     정당인  이화여자대학교 사회학과 졸업(복수전공 : 여성학)   
4   서울특별시 강남구 광평로47길     정치인                서울대학교 외교학과 졸업   

                                     경력 재산신고액(천원)       병역신고사항(본인) 납세실적(천원)  \
                                     경력 재산신고액(천원)       병역신고사항(본인)      납부액   
0               (전)더불어민주당 대표(전)국회의원(5선)    564787  군복무를 마치지 아니한 사람    46768   
1            (현)제38대 서울특별시장(전)제16대 국회의원   5902262        군복무를 마친사람   166766   
2            (현)서울시의원(전)아시아나항공 노동조합 위원장    735077        해당없음(비대상)     2478   
3   (전)2021 서울특별시장 보궐선거 후보(현)기본소득당 상임대표     31000        해당없음(비대상)      409   
4  (현)아리랑당 창당추진위원회 대표(전)아리랑당 창당준비위원회 대표    206189        군복무를 마친사람    26509   

                전과기록유무(건수) 입후보횟수  
  최근5년간체납액 현체납액 전과기록유무(건수) 입후보횟수  
0        0    0         2건    8회  
1        0    0         없음    6회  
2        0    0         1건    1회  
3        0    0         없음    5회  
4        0    0         없음    6회

In [81]:
ecode = ['3', '4', '5', '6', '8', '9', '11', '10', '2']
ename= ['시·도지사선거',
 '구·시·군의 장선거',
 '시·도의회의원선거',
 '구·시·군의회의원선거',
 '광역의원비례대표선거',
 '기초의원비례대표선거',
 '교육감선거',
 '교육의원선거',
 '국회의원선거']

In [133]:
ecode_name = dict(zip(ecode, ename))
ecode_name

{'3': '시·도지사선거',
 '4': '구·시·군의 장선거',
 '5': '시·도의회의원선거',
 '6': '구·시·군의회의원선거',
 '8': '광역의원비례대표선거',
 '9': '기초의원비례대표선거',
 '11': '교육감선거',
 '10': '교육의원선거',
 '2': '국회의원선거'}

In [153]:
def get_city_page(electionCode, city_code):
    url_xhtml = "http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml"
    params = "electionId=0020220601&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020220601%2Fcp%2Fcpri03.jsp"
    params = params + "&topMenuId=CP&secondMenuId=CPRI03&menuId=CPRI03&statementId=CPRI03_%233"
    params = f"{params}&electionCode={electionCode}&cityCode={city_code}&proportionalRepresentationCode=0&townCode=-1&dateCode=0&x=73&y=23"

    df_temp = pd.DataFrame()
    response = requests.post(url_xhtml, params=params)
    df_temp = pd.read_html(response.text)[0]
    df_temp["선거구분"] = ecode_name[electionCode]
    df_temp["선거구"] = city_dict[city_code]
    return df_temp

In [142]:
for eckey in ecode_name:
    print(eckey, ecode_name[eckey])

3 시·도지사선거
4 구·시·군의 장선거
5 시·도의회의원선거
6 구·시·군의회의원선거
8 광역의원비례대표선거
9 기초의원비례대표선거
11 교육감선거
10 교육의원선거
2 국회의원선거


In [145]:
city_temp_list = []
for electionCode in ecode_name:
    for city_code in city_dict:
        try:
            temp = get_city_page(electionCode, city_code)
            city_temp_list.append(temp)
        except:
            continue

In [146]:
city_temp_list

[    선거구명  사진 기호   소속정당명          성명(한자) 성별         생년월일(연령)  \
     선거구명  사진 기호   소속정당명          성명(한자) 성별         생년월일(연령)   
 0  서울특별시 NaN  1  더불어민주당  송영길(宋永吉)  공약보기  남  1963.03.21(59세)   
 1  서울특별시 NaN  2    국민의힘  오세훈(吳世勲)  공약보기  남  1961.01.04(61세)   
 2  서울특별시 NaN  3     정의당  권수정(權秀靜)  공약보기  여  1973.08.30(48세)   
 3  서울특별시 NaN  4   기본소득당  신지혜(申智惠)  공약보기  여  1987.10.22(34세)   
 4  서울특별시 NaN  5     무소속  김광종(金光宗)  공약보기  남  1962.07.25(59세)   
 
                   주소      직업                           학력  \
                   주소      직업                           학력   
 0    서울특별시 송파구 거마로3길     변호사           연세대학교 상경대학 경영학과 졸업   
 1    서울특별시 광진구 능동로4길  서울특별시장         고려대학교 법학대학원 졸업(법학박사)   
 2  서울특별시 강서구 강서로56나길   서울시의원                충남대학교 물리학과 졸업   
 3   서울특별시 은평구 통일로72길     정당인  이화여자대학교 사회학과 졸업(복수전공 : 여성학)   
 4   서울특별시 강남구 광평로47길     정치인                서울대학교 외교학과 졸업   
 
                                      경력 재산신고액(천원)       병역신고사항(본인) 납세실적(천원)  \
                                  

In [147]:
df_all = pd.concat(city_temp_list)

In [151]:
df_all.to_csv("후보자명부.csv")

In [152]:
df_all.shape

(495, 22)